In [1]:
import pandas as pd
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.dates as md

from itertools import count
from collections import Counter
import matplotlib

from nltk.tokenize import RegexpTokenizer


from time import mktime
from datetime import datetime
from datetime import timedelta


The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
import sys

def parse(f,chunksize = 100000, cols = None, test_bool = False, dtype = None):
    lines = 9041*1000
    for i,gm_chunk in enumerate(pd.read_csv("../ira_tweets_csv_hashed.csv",chunksize = chunksize, usecols = cols, dtype = dtype)):
        f(gm_chunk)
        if test_bool:
            print('Done')
            break
        j = i*chunksize/lines
        sys.stdout.write('\r')
        # the exact output you're looking for:
        bar_int = int(j*20+1)
        sys.stdout.write("[%-20s] %d%%" % ('='*bar_int, int(j*100+1)))
        sys.stdout.flush()

In [3]:
global df_test
def f(chunk):
    global df_test
    df_test = chunk

parse(f, 2, test_bool = True)

df_test[:1].to_string()

FileNotFoundError: File b'../ira_tweets_csv_hashed.csv' does not exist

# Sentiment Analysis

In [ ]:
#import the dataset
df_labmt = pd.read_csv("Data_Set_S1.txt", delimiter = '\t', header = 2)

In [ ]:
#create a dictionary with the words as keys
#and their happiness_average as values
havg_w = {w:v for w,v in zip(df_labmt['word'], df_labmt['happiness_average'])}

In [ ]:
#function that returns the sentiment of a wordlist
#if the wordlist is empty, than it will return
#the type np.nan, which than wont be incorporated for calculating the mean

#SENTIMENTS
def h_avg(wordlist):
    if len(wordlist) == 0:
        return np.nan
    else:
      #np.nanmean calculates the mean without incorporating np.nan
        return np.nanmean([havg_w.get(w, np.nan) for w in wordlist])

In [ ]:
num_lines = 0

def f(chunk):
    global num_lines
    num_lines = num_lines + len(chunk)
    
parse(f, 1000000, test_bool = False)
num_lines

In [ ]:
cols = ['tweet_language', 'account_language']

langs = set()

def f(chunk):
    global langs
    for t,a in zip(chunk['tweet_language'], chunk['account_language']):
        langs.add(t)
        langs.add(a)
        
parse(f, 1000000, cols = cols, test_bool = False)

# creating large array 'sents' with all important data

(including sentiments of tweets and profile desciption)

In [ ]:
langs_dict = {l:i for i,l in enumerate(list(langs))}

In [ ]:
min_times = datetime(2009, 5, 9, 15, 59)
min_times

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')

to_date_time = lambda t: datetime.strptime(t, '%Y-%m-%d %H:%M')


cols = ['tweet_text', 'tweet_language', 'like_count', 'retweet_count', 'quote_count', 'reply_count', 'account_language', 'user_profile_description', 'tweet_time', 'follower_count']

k = 0

sents = np.zeros((num_lines + 1, 12))
#tweets = []


def f(chunk):
    global k
    global sents
    
    for t,lang,content,likes,retw,quotes,replys,ac_lang,desc, fol_count in zip(chunk['tweet_time'],chunk['tweet_language'], chunk['tweet_text'], chunk['like_count'], chunk['retweet_count'], chunk['quote_count'], chunk['reply_count'], chunk['account_language'], chunk['user_profile_description'], chunk['follower_count']):
        k = k + 1
        sents[k,0] = (to_date_time(t) - min_times).days
        sents[k,1] = langs_dict[lang]
        if type(content) != float and len(content) > 0:
            words = [w.lower() for w in tokenizer.tokenize(content)]
            sents[k,2] = len(words)
            sents[k,3] = h_avg(words)
            
        sents[k,4] = likes
        sents[k,5] = retw
        sents[k,6] = quotes
        sents[k,7] = replys
            
        sents[k,8] = langs_dict[ac_lang]
        if type(desc) != float and len(desc) > 0:
            words = [w.lower() for w in tokenizer.tokenize(desc)]
            sents[k,9] = h_avg(words)
            sents[k,10] = len(words)
        
        sents[k,11] = fol_count
            
        

        
        #tweets.extend([content])

parse(f, 50000, cols = cols, test_bool = False)

## save this array

In [ ]:
np.save("tweet_sentiments.npy", sents)

In [ ]:
sents = np.load('tweet_sentiments.npy')

In [ ]:
def clean_nan_zeros(raw):
    clean = raw[~np.isnan(raw)]
    clean = clean[clean != 0]
    return clean

In [ ]:
def clean_sents(raw_sents):
    cleaned_sents = clean_nan_zeros(raw_sents)
    cleaned_sents = cleaned_sents[cleaned_sents < 10]
    return cleaned_sents

In [ ]:
tw_sents = clean_sents(sents[:,3])
desc_sents = clean_sents(sents[:,9])

In [ ]:
len(tw_sents), len(desc_sents)

In [ ]:
np.average(tw_sents), np.std(tw_sents)

In [ ]:
np.average(desc_sents), np.std(desc_sents)

In [ ]:
plt.hist(tw_sents)

In [ ]:
plt.hist(desc_sents)

In [ ]:
#bins = np.linspace(0,10,1000)
tw_hist, tw_bins = np.histogram(tw_sents, bins = np.linspace(0.01,10, 500))
desc_hist, desc_bins = np.histogram(desc_sents, bins = np.linspace(0.01,10,100))

In [ ]:
plt.plot(tw_bins[1:], tw_hist, '.', color = 'blue')
plt.plot(desc_bins[1:], desc_hist, '.', color = 'green' )
plt.ylim((0,0.5e6))

# Comparing sentiments of tweets (blue) and profiles (green)

In [ ]:
plt.plot(tw_bins[1:], np.log(tw_hist), '.', color = 'blue', alpha = 0.3)
plt.plot(desc_bins[1:], np.log(desc_hist), '.', color = 'green', alpha = 0.3)

# Heatmap: tweet-Sentiments to profile description sentiments

In [ ]:
raw_tw_sents = sents[:,3]
raw_desc_sents = sents[:,9]

In [ ]:
bins_x = np.linspace(0,10, 100)
bins_y = np.linspace(0,10,1000)

bins = np.meshgrid(bins_x, bins_y)
heatmap, xedges, yedges = np.histogram2d(raw_desc_sents, raw_tw_sents, bins=[100,100], range = [[0, 10], [0, 10]])
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.clf()
plt.imshow(np.log(heatmap).T, extent=extent, origin='lower')

plt.savefig('tw_ents-desc_sents.png', dpi = 300)
plt.show()

# Like distribution according to sentiments

## Like distribution

In [ ]:
a = plt.hist(clean_nan_zeros(sents[:,4]), bins = np.linspace(0,100, 100))

In [ ]:
bins_x = np.linspace(0,10, 100)
bins_y = np.linspace(0,10,1000)

heatmap, xedges, yedges = np.histogram2d(raw_tw_sents, sents[:,4], bins=[100,100], range = [[0, 10], [0, 1000]])
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.clf()
plt.imshow(np.log(heatmap).T, extent=extent, origin='lower')
plt.axis('auto')

plt.savefig('sent-like-dist.png', dpi = 300)
plt.show()

In [ ]:
likes_avg_sents = [np.average(heatmap[i]) for i in range(len(heatmap))]

In [ ]:
plt.plot(likes_avg_sents, '.')

In [ ]:
hist2,bins2 = np.histogram(sents_over2, bins = np.linspace(0,10,100))

In [ ]:
plt.plot(bins[1:], np.log(hist), '.', alpha = 0.5, color = 'orange')
plt.plot(bins2[1:], np.log(hist2), 'x', color = 'black', ms = 2, alpha = 0.5)

In [ ]:
a_hist = np.argsort(hist)
hist[a_hist[::-1]], bins[a_hist[::-1]]

In [ ]:
np.average(cleaned_sents)

In [ ]:
np.histogram(sents[:,0], bins = [0,10,100,1e17])

In [ ]:
h_avg(["hi", "this", "is", "jöaldfj", "no"])